In [9]:
#!pip install pandas
#!pip install bokeh
#!pip install sklearn
#!pip install v3io_frames

import pandas as pd
import pickle
import sklearn
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask_ml.wrappers import ParallelPostFit
import os
import v3io_frames as v3f

## Load & Parallelize model

In [2]:
MODEL_FILENAME='/v3io/bigdata/netops_models/netops.model'
with open(MODEL_FILENAME, 'rb+') as f:
    model = pickle.load(f) 
    
clf = ParallelPostFit(estimator=model,
                      scoring='accuracy')

## Load sample data

In [10]:
client = v3f.Client('framesd-h7a26fh1yk-ncymt:8081', password=os.environ['V3IO_PASSWORD'])

In [20]:
df = dd.from_pandas(client.read(backend='tsdb', query='select avg(cpu_utilization) as cpu_utilization, avg(latency) as latency, avg(packet_loss) as packet_loss, avg(throughput) as throughput from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='1h').reset_index(), npartitions=4).drop(['time', 'company', 'data_center', 'device'], axis=1)
df.head(5)

/User/.pythonlibs/lib/python3.6/site-packages/v3io_frames/pdutils.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  ignore_index=True,
/User/.pythonlibs/lib/python3.6/site-packages/dask/dataframe/core.py:4479: UserWarning: Insufficient elements for `head`. 5 elements requested, only 4 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,cpu_utilization,latency,packet_loss,throughput
0,73.835210,11.459141,4.809160,232.458034
1,73.988361,11.762500,5.525210,230.131261
2,75.600191,15.708626,6.765267,219.776107
3,73.729097,12.588445,5.018908,228.279349


## Predict

In [22]:
clf.predict(df.head(2))

array([0., 0.])